Import necessary packages.

In [1]:
import numpy as np
import pandas as pd
from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping
from keras.utils import to_categorical

Using TensorFlow backend.


Read in the final dataset, with all of our variables.

In [2]:
df = pd.read_csv('full_dataset.csv', encoding = "ISO-8859-1", index_col=0)

C:\Users\seanc_000\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


For this version of the neural net, we decided to keep only the binary variables for tfidf and the hashtag variables, in addition to the negative sentiment and compound sentiment variables as predictors. These are variables 11 through 1576. Create a dataframe out of only these columns below. To get an idea of what this looks like, uncomment the print statement.

In [3]:
rel_var = df.iloc[:,11:1576]
print(rel_var.head())

                    sorri  celin  bangladesh  mapl  honestli  femin  ref  \
ID                                                                         
319399851215433729      0      0           0     0         0      0    0   
320817818222358529      0      0           0     0         0      0    0   
324114200450437120      0      0           0     0         0      0    0   
326286656854454273      0      0           0     0         0      0    0   
381988216292655104      0      0           0     0         0      0    0   

                    footbal  kitchen  cunt    ...     'needarethinkinformat'  \
ID                                            ...                              
319399851215433729        0        0     0    ...                          0   
320817818222358529        0        0     0    ...                          0   
324114200450437120        0        0     0    ...                          0   
326286656854454273        0        0     0    ...                  

Because keras requires a numpy array as input, we convert the dataframe of our predictors to a numpy array below. To see what this looks like, uncomment the print statement.

In [4]:
predictors = pd.DataFrame.as_matrix(rel_var, columns=None)
#print(predictors)

Here, we change our class variables to numeric values for the keras package to predict. To compare the lists, uncomment the print statement. We then use keras's to_categorical function on the numeric values to show that we're predicting multiple classes. n_cols is the number columns in the predictors dataframe, which will be fed as an argument to the model.add() function below

In [5]:
new_class = []

for i in df.Class:
    if i == 'sexism':
        new_class.append(0)
    elif i == 'racism':
        new_class.append(1)
    else:
        new_class.append(2)

#print(list(df.Class)[0:10])
#print(new_class[0:10])

target=to_categorical(new_class)
n_cols = predictors.shape[1]

Here we define the architecture of the model. Our model is a sequential model, and each model.add() line represents a layer in the neural network; the number after Dense( represents the number of nodes in that layer. Our activation function is "relu" for each layer, and "softmax" for the last layer (because it's predicting a categorical variable, this tells it to predict whichever class has the highest probability of being true). 

In [6]:
#Create model
model = Sequential()

#Add layers to model
model.add(Dense(100, activation='relu', input_shape = (n_cols,)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(3, activation='softmax'))

Finally, we run and compile the code. We use the adam optimizer, a loss function of categorical crossentropy (used for categorical target variables), and our metric of interest is accuracy. We also add an early stopping function with a patience of two, meaning that if our model fails to improve on our metric for two epochs, the model should stop running. The model fit function is the final piece of important code. It takes as input our predictors, our target variable, a validation split (what proportion of the dataset should be set aside for testing), and the number of epochs (this can be higher since we have an early stopping function). The accuracy of the model is the highest number under "val_acc." 

In [7]:
#compile 
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

early_stop = EarlyStopping(patience=2) 

#fit model
model.fit(predictors, target, validation_split=0.3, epochs=20, callbacks=[early_stop])

Train on 11294 samples, validate on 4841 samples
Epoch 1/20
11294/11294 [==============================] - 115s 10ms/step - loss: 0.7726 - acc: 0.6947 - val_loss: 0.6735 - val_acc: 0.7744
Epoch 2/20
11294/11294 [==============================] - 116s 10ms/step - loss: 0.6815 - acc: 0.7388 - val_loss: 0.6172 - val_acc: 0.7922
Epoch 3/20
11294/11294 [==============================] - 136s 12ms/step - loss: 0.6510 - acc: 0.7482 - val_loss: 0.6468 - val_acc: 0.7693
Epoch 4/20
11294/11294 [==============================] - 135s 12ms/step - loss: 0.6352 - acc: 0.7552 - val_loss: 0.6507 - val_acc: 0.7852
